<a href="https://colab.research.google.com/github/Sparus42/2021IntelON-RL/blob/main/Cityflow_Gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cloning Repositories and Installations

In [1]:
!sudo apt update && sudo apt install -y build-essential cmake
#!git clone https://github.com/IntelLabs/coach.git
!git clone https://github.com/cityflow-project/CityFlow.git
!git clone https://github.com/Sparus42/2021IntelON-RL.git
#%cd /content/coach
#!pip install .
%cd /content/CityFlow
!pip install .
%cd /content/2021IntelON-RL
!pip install .
!pip install gym numpy sigopt stable_baselines3

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [419 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [848 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,085 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:13 h

# SigOpt Hyperparameter Tuning

Connects SigOpt to web API

In [ ]:
! sigopt config
from sigopt import Connection
conn = Connection()

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
SigOpt API token (find at https://app.sigopt.com/tokens/info): ADPRWOGTWNUWJFWDMJVQMMPUQPQLJBZVJPQOGFGBLSMOGBAB
Log Collection
	This will capture and upload the standard output and standard error of your
	Runs from the CLI and notebook cells so that you can view them on the SigOpt dashboard.
Enable log collection [y/N]: y
Notebook Cell Tracking
	This will record and upload the content of your notebook cells so that you can view them
	on the SigOpt dashboard.
Enable cell tracking [y/N]: y


Sets up SigOpt Experiment for Hyperparameter Tuning

In [ ]:
from sigopt import Connection as sigopt
from torch import nn as nn

experiment = conn.experiments().create(
  name="Jinan 3x4 A2C Optimization",
  type="offline",
  parameters=[
              #dict(name="batch_size", type="int", grid=[8, 16, 32, 64, 128, 256, 512]),
              dict(name="n_steps", type="int", grid=[8, 16, 32, 64, 128, 256, 512, 1024, 2048]),
              dict(name="gamma", type="double", grid=[0.9, 0.95, 0.98, 0.99, 0.995, 0.999, 0.9999]),
              dict(name="learning_rate", type="double", transformation="log", bounds=dict(min=1e-5, max=1)),
              dict(name="ent_coef", type="double", transformation="log", bounds=dict(min=0.00000001, max=.01)),
              #dict(name="clip_range", type="double", grid=[0.1, 0.2, 0.3, 0.4]),
              #dict(name="n_epochs", type="int", grid=[1, 5, 10, 20]),
              dict(name="gae_lambda", type="double", grid=[0.8, 0.9, 0.92, 0.95, 0.98, 0.99, 1.0]),
              dict(name="max_grad_norm", type="double", grid=[0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 5]),
              dict(name="vf_coef", type="double", bounds=dict(min=0.0, max=1.0)),
              dict(name="activation_fn", type="categorical", categorical_values=['tanh', 'relu']),
              dict(name="normalize_advantage", type="categorical", categorical_values=['False', 'True']),
              dict(name="ortho_init", type="categorical", categorical_values=['False', 'True']),
              dict(name="use_rms_prop", type="categorical", categorical_values=['False', 'True']),
              dict(name="policy_layer_num", type="int", bounds=dict(min=1, max=4)),
              dict(name="val_layer_num", type="int", bounds=dict(min=1, max=4)),
              dict(name="policy_layer_width", type="int", grid=[64, 128, 256, 512]),
              dict(name="val_layer_width", type="int", grid=[64, 128, 256, 512])
  ],
  metrics=[dict(name="reward", strategy="optimize", objective="maximize")],
  parallel_bandwidth=1,
  observation_budget=150
)
print(experiment.id)

440540


Run the SigOpt Experiment

In [ ]:
from torch import nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import gym
import gym_cityflow
import CityFlow
import numpy as np
from stable_baselines3 import A2C
from distutils import util

for _ in range(200-23):
  #Pull suggestion from SigOpt
  suggestion_full = conn.experiments(experiment.id).suggestions().create()
  suggestion = suggestion_full.assignments

  #set up network
  net_arch=[dict(pi=[suggestion["policy_layer_width"]] * suggestion["policy_layer_num"],
                 vf=[suggestion["val_layer_width"]] * suggestion["val_layer_num"])]

  #set activation fn
  suggestion["activation_fn"] = {"tanh": nn.Tanh, "relu": nn.ReLU, "elu": nn.ELU, "leaky_relu": nn.LeakyReLU}[suggestion["activation_fn"]]

  #change vals to bool
  suggestion["normalize_advantage"] = bool(util.strtobool(suggestion["normalize_advantage"]))
  suggestion["ortho_init"] = bool(util.strtobool(suggestion["ortho_init"]))
  suggestion["use_rms_prop"] = bool(util.strtobool(suggestion["use_rms_prop"]))

  #define keys for the policy and model
  policy_keys = ["activation_fn", "ortho_init"]
  model_keys = ["n_steps", "gamma", "learning_rate", "ent_coef", "normalize_advantage", "use_rms_prop", "gae_lambda", "max_grad_norm", "vf_coef",]

  policy_kwargs = { policy_key: suggestion[policy_key] for policy_key in policy_keys }
  policy_kwargs["net_arch"] = net_arch

  model_kwargs = { model_key: suggestion[model_key] for model_key in model_keys }

  print(policy_kwargs)
  print("\n")
  print(model_kwargs)
  print("\n")

  env = gym.make('gym_cityflow:CityFlow-Jinan-3x4-v0')
  env.set_save_replay(False)

  model = A2C('MlpPolicy', env, verbose=1, **model_kwargs, policy_kwargs=policy_kwargs)
  model.learn(total_timesteps=30000)

  eval_env = gym.make('gym_cityflow:CityFlow-Jinan-3x4-v0')
  mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=1, deterministic=True)

  #print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

  conn.experiments(experiment.id).observations().create(
    suggestion=suggestion_full.id,
    value=mean_reward
  )

# get the best Runs for the Experiment
#best_runs = experiment.get_best_runs()

{'activation_fn': <class 'torch.nn.modules.activation.Tanh'>, 'ortho_init': False, 'net_arch': [{'pi': [512, 512], 'vf': [512, 512]}]}


{'n_steps': 64, 'gamma': 0.9, 'learning_rate': 0.0014930726144836766, 'ent_coef': 0.006263898299777303, 'normalize_advantage': False, 'use_rms_prop': True, 'gae_lambda': 1.0, 'max_grad_norm': 0.6, 'vf_coef': 0.2440564543556722}


/content/2021IntelON-RL/gym_cityflow/envs/Jinan_3x4_config
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.6e+03   |
|    ep_rew_mean        | -5.11e+06 |
| time/                 |           |
|    fps                | 264       |
|    iterations         | 100       |
|    time_elapsed       | 24        |
|    total_timesteps    | 6400      |
| train/                |           |
|    entropy_loss       | -17.2     |
|    explained_variance | 5.96e-08  |
|    learning_rate     

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


{'activation_fn': <class 'torch.nn.modules.activation.ReLU'>, 'ortho_init': False, 'net_arch': [{'pi': [512, 512, 512, 512], 'vf': [512, 512, 512]}]}


{'n_steps': 2048, 'gamma': 0.995, 'learning_rate': 2.46196726100607e-05, 'ent_coef': 0.00013171246153275985, 'normalize_advantage': False, 'use_rms_prop': True, 'gae_lambda': 0.92, 'max_grad_norm': 0.9, 'vf_coef': 0.25007989482268567}


/content/2021IntelON-RL/gym_cityflow/envs/Jinan_3x4_config
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
/content/2021IntelON-RL/gym_cityflow/envs/Jinan_3x4_config
{'activation_fn': <class 'torch.nn.modules.activation.Tanh'>, 'ortho_init': True, 'net_arch': [{'pi': [512, 512], 'vf': [128]}]}


{'n_steps': 256, 'gamma': 0.98, 'learning_rate': 0.2146803815771016, 'ent_coef': 1.0414158122052657e-07, 'normalize_advantage': True, 'use_rms_prop': True, 'gae_lambda': 1.0, 'max_grad_norm': 0.7, 'vf_coef': 0.9308530929637302}


/content/2021IntelON-RL/gym_cityflow/

/usr/local/lib/python3.7/dist-packages/stable_baselines3/a2c/a2c.py:165: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  th.nn.utils.clip_grad_norm_(self.policy.parameters(), self.max_grad_norm)


Streaming output truncated to the last 5000 lines.
|    std                | 1         |
|    value_loss         | 3.39e+07  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.6e+03   |
|    ep_rew_mean        | -4.6e+06  |
| time/                 |           |
|    fps                | 227       |
|    iterations         | 700       |
|    time_elapsed       | 98        |
|    total_timesteps    | 22400     |
| train/                |           |
|    entropy_loss       | -17       |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 1.53e-05  |
|    n_updates          | 699       |
|    policy_loss        | -5.87e+04 |
|    std                | 1         |
|    value_loss         | 1.26e+07  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.6e+03   |
|    ep_rew_mean        | -4.6e+06  |

KeyboardInterrupt: ignored

# Stable Baselines 3 Model

To use GPU/TPU: Runtime > Change runtime type

Stable Baselines 3 docs here, feel free to mess with other models, policies, and settings: https://stable-baselines3.readthedocs.io/en/master/index.html


---



Connects SigOpt to web API

In [ ]:
#! sigopt config
from sigopt import Connection
conn = Connection(client_token='ADPRWOGTWNUWJFWDMJVQMMPUQPQLJBZVJPQOGFGBLSMOGBAB')

/usr/local/lib/python3.7/dist-packages/sigopt/interface.py:325: UserWarning: You're currently using the old SigOpt Experience. Try out the new and improved SigOpt experience by getting started with the docs today. You have until July 2022 to migrate over without experiencing breaking changes.
  UserWarning,


Imports libraries and creates environment.

In [ ]:
import gym
import gym_cityflow
import CityFlow
import numpy as np
from stable_baselines3 import A2C

### Create environment
env = gym.make('gym_cityflow:CityFlow-Jinan-3x4-v0')
#env = gym.make('gym_cityflow:CityFlow-1x1-LowTraffic-v0')
env.set_save_replay(False)

"""print(env.action_space)
print("\n")
print(env.observation_space)
print(np.shape(env.observation_space))"""

/content/2021IntelON-RL/gym_cityflow/envs/Jinan_3x4_config


'print(env.action_space)\nprint("\n")\nprint(env.observation_space)\nprint(np.shape(env.observation_space))'

Train model using SigOpt's recommended parameters

In [ ]:
from torch import nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import gym
import gym_cityflow
import CityFlow
import numpy as np
from stable_baselines3 import A2C
from distutils import util

#Pull suggestion from SigOpt
best_assignments = (
  conn.experiments(440540).best_assignments().fetch().data[0]
)
suggestion = best_assignments.assignments

#set up network
net_arch=[dict(pi=[suggestion["policy_layer_width"]] * suggestion["policy_layer_num"],
                vf=[suggestion["val_layer_width"]] * suggestion["val_layer_num"])]

#set activation fn
suggestion["activation_fn"] = {"tanh": nn.Tanh, "relu": nn.ReLU, "elu": nn.ELU, "leaky_relu": nn.LeakyReLU}[suggestion["activation_fn"]]

#change vals to bool
suggestion["normalize_advantage"] = bool(util.strtobool(suggestion["normalize_advantage"]))
suggestion["ortho_init"] = bool(util.strtobool(suggestion["ortho_init"]))
suggestion["use_rms_prop"] = bool(util.strtobool(suggestion["use_rms_prop"]))

#Define keys for the policy and model
policy_keys = ["activation_fn", "ortho_init"]
model_keys = ["n_steps", "gamma", "learning_rate", "ent_coef", "normalize_advantage", "use_rms_prop", "gae_lambda", "max_grad_norm", "vf_coef",]

policy_kwargs = { policy_key: suggestion[policy_key] for policy_key in policy_keys }
policy_kwargs["net_arch"] = net_arch

model_kwargs = { model_key: suggestion[model_key] for model_key in model_keys }

#Create and train model
model = A2C('MlpPolicy', env, verbose=1, **model_kwargs, policy_kwargs=policy_kwargs)
model.learn(total_timesteps=5000000)

/usr/local/lib/python3.7/dist-packages/sigopt/objects.py:340: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  RuntimeWarning,


Streaming output truncated to the last 5000 lines.
| time/                 |           |
|    fps                | 369       |
|    iterations         | 50400     |
|    time_elapsed       | 8734      |
|    total_timesteps    | 3225600   |
| train/                |           |
|    entropy_loss       | -17.8     |
|    explained_variance | 0.904     |
|    learning_rate      | 0.000154  |
|    n_updates          | 50399     |
|    policy_loss        | 0.284     |
|    std                | 1.07      |
|    value_loss         | 1.91e+03  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.6e+03   |
|    ep_rew_mean        | -2.08e+06 |
| time/                 |           |
|    fps                | 369       |
|    iterations         | 50500     |
|    time_elapsed       | 8751      |
|    total_timesteps    | 3232000   |
| train/                |           |
|    entropy_loss       | -17.8     |

In [ ]:
model.learn(total_timesteps=3000000)

Streaming output truncated to the last 5000 lines.
| time/                 |           |
|    fps                | 385       |
|    iterations         | 19100     |
|    time_elapsed       | 3174      |
|    total_timesteps    | 1222400   |
| train/                |           |
|    entropy_loss       | -18.5     |
|    explained_variance | 0.856     |
|    learning_rate      | 0.000154  |
|    n_updates          | 97224     |
|    policy_loss        | 0.271     |
|    std                | 1.13      |
|    value_loss         | 1.18e+04  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.6e+03   |
|    ep_rew_mean        | -1.99e+06 |
| time/                 |           |
|    fps                | 385       |
|    iterations         | 19200     |
|    time_elapsed       | 3191      |
|    total_timesteps    | 1228800   |
| train/                |           |
|    entropy_loss       | -18.5     |

Save the model to a file.

In [ ]:
model.save("a2c_Jinan")

Load the model from a file.

In [ ]:
model.load("a2c_Jinan")

Evaluate the model.

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

eval_env = gym.make('gym_cityflow:CityFlow-Jinan-3x4-v0')
#eval_env.set_replay_path("aaaaa.txt")
#eval_env = gym.make('gym_cityflow:CityFlow-1x1-LowTraffic-v0')
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=2, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/content/2021IntelON-RL/gym_cityflow/envs/Jinan_3x4_config


/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward=-5948878.00 +/- 0.0


To view recordings:


1.   Clone the CityFlow git (https://github.com/cityflow-project/CityFlow.git) onto your computer
2. Enter your local CityFlow directory and open frontend/index.html
3.   Open file menu on left of Colab after evaluating the model
4.   Browse to /content/2021IntelON-RL/gym_cityflow/envs/[env name]
5.   Right click and download replay.txt and roadnet.json
6.   Load the downloaded files into the CityFlow index.html

**Don't download the training replays unless you want to watch the *entire* training process in real time! Evaluate the model first to see the final result.**



# Coach Model (Ignore)

In [ ]:
# Coach needs this to work for some reason, ignore
!pip uninstall -y tensorflow
!pip install tensorflow

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
ERROR: Operation cancelled by user
     |███████████▎                    | 161.7 MB 1.5 MB/s eta 0:03:20

In [ ]:
#

from rl_coach.agents.clipped_ppo_agent import ClippedPPOAgentParameters
from rl_coach.environments.gym_environment import GymEnvironmentParameters
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import SimpleSchedule
from rl_coach.architectures.embedder_parameters import InputEmbedderParameters
from rl_coach.filters.filter import NoInputFilter, NoOutputFilter
from rl_coach.filters.observation import ObservationNormalizationFilter
import CityFlow
import gym
import gym_cityflow
import tensorflow as tf

# Resetting tensorflow graph as the network has changed.
tf.compat.v1.reset_default_graph()

# define the environment parameters
env_params = GymEnvironmentParameters(level='CityFlow-Jinan-3x4-v0')
env_params.default_input_filter = NoInputFilter()
env_params.default_output_filter = NoOutputFilter()

# Clipped PPO
agent_params = ClippedPPOAgentParameters()
agent_params.network_wrappers['main'].input_embedders_parameters = {
    'state': InputEmbedderParameters(scheme=[]),
    'desired_goal': InputEmbedderParameters(scheme=[])
}

graph_manager = BasicRLGraphManager(
    agent_params=agent_params,
    env_params=env_params,
    schedule_params=SimpleSchedule()
)

In [ ]:
#Change tf.variable_scope to tf.compat.v1.variable_scope manually for now
graph_manager.improve()

Starting to improve simple_rl_graph task index 0


IndexError: ignored